# Aristeia Calculator

In [5]:
# Import die radio widget
from collections import defaultdict
import numpy as np
import pandas as pd
import holoviews as hv
hv.extension('bokeh', logo=False)

from die import die
from die_widget import create_die_widgets, display_widgets

In [6]:
# Instantiate die 
# TODO: A "Roll" class would be a great way to introduce Aaron to OOP
a = create_die_widgets()
d = create_die_widgets()

### Attacker

In [7]:
display_widgets(a)

HBox(children=(RadioButtons(description=u'Black', options=(0, 1, 2, 3), value=0), RadioButtons(description=u'Blue', options=(0, 1, 2, 3), value=0), RadioButtons(description=u'Green', options=(0, 1), value=0), RadioButtons(description=u'Orange', options=(0, 1, 2, 3), value=0), RadioButtons(description=u'Red', options=(0, 1), value=0), RadioButtons(description=u'Yellow', options=(0, 1, 2, 3), value=0)))

### Defender

In [170]:
display_widgets(d)

HBox(children=(RadioButtons(description=u'Black', options=(0, 1, 2, 3), value=0), RadioButtons(description=u'Blue', options=(0, 1, 2, 3), value=0), RadioButtons(description=u'Green', options=(0, 1), value=0), RadioButtons(description=u'Orange', options=(0, 1, 2, 3), value=0), RadioButtons(description=u'Red', options=(0, 1), value=0), RadioButtons(description=u'Yellow', options=(0, 1, 2, 3), value=0)))

In [171]:
def roll_die_pool(die, counts):
    dists = defaultdict(list)
    for choice, num_die in counts.iteritems():
        for _ in xrange(num_die):
            for glyph, value in die[choice].iloc[np.random.randint(0, 6)].iteritems():
                dists[glyph].append(value)
    return pd.DataFrame(dists)

def rolls_expected_value(rolls):
    return pd.concat(rolls).sum().divide(len(rolls))

def rolls_to_dists(rolls):
    return pd.concat([x.sum() for x in rolls], axis=1).T

def roll(die, attacker, defender, n_rolls=10000):
    # TODO: Add dynamic roll storage. Precheck for hash here, otherwise run algorithm

    # Objects to hold counts
    a_rolls, d_rolls = [], []
    t_total = defaultdict(list)
    output = {}  # Final output containing dists
    
    # Convert attacker / defender widgets to die counts
    a_counts = {x: y.value for x, y in attacker.iteritems()}
    d_counts = {x: y.value for x, y in defender.iteritems()}
    
    # Simulate n_rolls
    for _ in xrange(n_rolls):
        
        # Roll attacker pool
        a_pool = roll_die_pool(die, a_counts)
        a_raw_pool = a_pool.copy()  # We want to record an unadulterated set for attacker distributions
        
        # Roll defender pool
        d_pool = roll_die_pool(die, d_counts)

        # Calculate total diffs
        a = pd.Series({x: sum(y) for x, y in a_pool.iteritems()})
        d = pd.Series({x: sum(y) for x, y in d_pool.iteritems()})
        
        # If crit-defense die, drop roll with highest number of hits
        # TODO: Improve logic for removing most glyphs if tie
        # Doesn't handle crit attack die
        if d.crit_defense:
            a_pool = a_pool.drop(a_pool.hit.argmax())
            a = pd.Series({x: sum(y) for x, y in a_pool.iteritems()})
        
        a_diff = a.hit - d.defense if (a.hit - d.defense) > 0 else 0
        d_diff = d.hit - a.defense if (d.hit - a.defense) > 0 else 0
        
        # Add crits, which are unaffected by defense
        a_diff += a.crit
        
        # Update totals
        a_rolls.append(a_raw_pool.sum())
        d_rolls.append(d_pool.sum())
        t_total['a'].append(a_diff)
        t_total['d'].append(d_diff)
        
    return pd.concat(a_rolls, axis=1), pd.concat(d_rolls, axis=1), t_total

In [172]:
n_rolls = 100
%time a_rolls, d_rolls, diffs = roll(die, a, d, n_rolls=n_rolls)

CPU times: user 471 ms, sys: 110 ms, total: 581 ms
Wall time: 835 ms


In [173]:
a_rolls

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
crit,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
crit_defense,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
defense,1,0,1,1,2,1,1,0,2,2,...,1,0,1,2,2,1,0,0,0,0
hit,2,2,4,2,2,3,3,5,1,1,...,3,3,1,2,0,3,2,3,3,3
special,2,0,2,2,3,3,2,3,1,2,...,3,1,1,2,1,3,0,1,1,2


#### Calculate Expected Value

In [175]:
hv.Distribution(a_rolls.loc['hit'])

:Distribution   [hit]   (Density)

In [164]:
%%opts Bars [xrotation=45 tools=['hover']]
exp_a = a_rolls.sum(axis=1).divide(n_rolls)
exp_d = d_rolls.sum(axis=1).divide(n_rolls)
hv.Bars((exp_a.index, exp_a), label='Attacker') + \
hv.Bars((exp_d.index, exp_d), label='Defender')

:Layout
   .Bars.Attacker :Bars   [x]   (y)
   .Bars.Defender :Bars   [x]   (y)

#### Probability

In [162]:
hist = np.histogram(diffs['a'], bins=np.max(diffs['a']) + 1)
hist = hv.Histogram(hist, kdims='Damage Done', vdims='Probability Estimate')
# Normalize hist to probability (sum to 1)
freq = hist.data['Probability Estimate']
freq = np.array([float(i)/sum(freq) for i in freq])
hist.data['Probability Estimate'] = freq
hist

:Histogram   [Damage Done]   (Probability Estimate)

## User Display
TODO:

- Not shit histogram for damage done
- PMF from the not-shit histogram
- Provide expected values

In [117]:
hist = np.histogram(diffs['a'], bins=np.max(diffs['a']) + 1)
hist = hv.Histogram(hist)
# Normalize hist to probability (sum to 1)
freq = hist.data['Probability Estimate']
freq = np.array([float(i)/sum(freq) for i in freq])
hist.data['Probability Estimate'] = freq
hist

:Histogram   [Damage Done]   (Probability Estimate)

In [139]:
y, x = np.histogram(diffs['a'], bins=50)

In [140]:
hv.Curve((x, y))

:Curve   [x]   (y)

### PMF

0    4
1    1
Name: hit, dtype: int64

In [187]:
np.histogram(dists, 2)

(array([7, 3]), array([ 0.,  2.,  4.]))

In [195]:
# Convert rolls to dists
dists = rolls_to_dists(a_rolls)
frequencies, edges = np.histogram(dists.hit, 20)
# Make histogram
hist = hv.Histogram((frequencies, edges))
# Normalize hist to probability (sum to 1)
freq = hist.data['Frequency']
freq = np.array([float(i)/sum(freq) for i in freq])
hist.data['Frequency'] = freq

In [196]:
hist

:Histogram   [x]   (Frequency)